In [ ]:
import urllib3
import mysql.connector as sql
import certifi
from bs4 import BeautifulSoup

#Connection and Urllib3
db_connection = sql.connect(host='127.0.0.1', database='news_aggregator', user='root', password='')
cursor = db_connection.cursor()
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())

def generate_date(prev_date):
    from datetime import date, timedelta
    date_list = []
    for i in range(1, prev_date):
        tanggal = date.today() - timedelta(i)
        date_list.append(tanggal)
    
    return date_list

def insert(title, url, date_current):
    db_connection = sql.connect(host='127.0.0.1', database='news_aggregator', user='root', password='')
    cursor = db_connection.cursor()
    
    #Try Except
    try:
        data_table = ("INSERT INTO data (TITLE, URL, DATE) VALUES (%s, %s, %s)")
        data_value = (title, url, date_current.strftime('%Y-%m-%d'))

        cursor.execute(data_table, data_value)
        db_connection.commit()

    except sql.IntegrityError:
        print('duplicate entry')
        break
        
    cursor.close()
    db_connection.close()

In [ ]:
def pull_data_vemale(domain, pagination, duration):
    
    date = generate_date(duration)
    for date_current in date:
        for j in range(1, pagination):
            url = domain + str(j) + '?date=' + str(date_current.strftime('%m/%d/%Y'))
            print(url)
    
            #Get article from website
            req = http.request('GET', url)
            soup = BeautifulSoup(req.data, 'lxml')
            
            try:
                container = soup.find('ul', attrs={'id':'indeks-container'})
            except:
                print('indeks-container not found')
                break
                
            try:
                box = container.find_all('li')
            except:
                print('li not found')
                break

            for i in range(0, len(box)):
                box_title = container.find_all('li')[i]
                
                #Get Title Article
                try:
                    title = box_title.find('h2').text
                except:
                    print('h2.text not found')
                    break

                #Get URL Article
                try:
                    url = box_title.find('a').get('href')
                except:
                    print('href not found')
                    break

                #Try Except
                try:
                    data_table = ("INSERT INTO data (TITLE, URL, DATE) VALUES (%s, %s, %s)")
                    data_value = (title, url, date_current.strftime('%Y-%m-%d'))

                    cursor.execute(data_table, data_value)
                    db_connection.commit()

                except sql.IntegrityError:
                    print('duplicate entry')
                    break

    return "Done"